<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3c4349d48e9fdf6062b6ccb4f4ff964902282ffe9b88a33e281c832b5221c42f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 09:43:12
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.43 C
-------------------
Today PnL: -3.96 K (-0.03%)
Current PnL: -16.81 L (-11.31%)
CY Booked + Current PnL: -5.37 L (-3.61%)
-------------------
Total profit:  2.05 L
Total loss:  -18.86 L
-------------------
Total Booked + Current PnL: 21.61 L (17.76%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (7.98%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 82.33 L (57.46%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 10.06%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,14.38,66.0,H-MC,3.48,197320.0,32650.0,10675.0,0.48,19.83,5.41,26.31,89.0,3.06,1.41,29.39,X40N,NTT,AC
76,TTKPRESTIG,770.00,102.13,51.0,M-SC,3.30,87076.0,-13701.0,13793.0,0.06,-13.60,15.84,0.09,245.0,-0.99,0.62,14.40,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,67.0,H-MC,8.61,112800.0,12296.0,16807.0,0.58,12.23,14.90,28.96,79.0,0.73,0.80,50.36,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.83,219632.0,9266.0,19525.0,0.04,4.40,8.89,13.68,37.0,0.47,1.57,21.66,XY25,NTT,REFINERIES
83,VOLTAS,1530.00,-0.24,52.0,H-MC,2.78,209370.0,17628.0,20120.0,-1.70,9.19,9.61,19.69,99.0,0.88,1.49,16.62,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1258.85,80.0,M-SC,6.42,184906.0,3763.0,41696.0,10.91,2.08,22.55,25.10,235.0,0.09,1.32,48.18,XY24,NTT,HEALTHCARE
78,UNIONBANK,163.00,-11.63,67.0,M-LC,7.06,159521.0,18681.0,25810.0,2.72,13.26,16.18,31.59,66.0,0.72,1.14,43.27,XY24,NTT,BANKS
79,UNITDSPR,1644.00,-10.95,60.0,H-LC,7.60,233318.0,-826.0,49440.0,2.13,-0.35,21.19,20.77,86.0,-0.02,1.66,6.54,X40N,NTT,BREWERIES
31,HINDZINC,730.22,23.40,63.0,M-LC,1.68,200644.0,-4432.0,116996.0,1.81,-2.16,58.31,54.89,52.0,-0.04,1.43,21.41,X5K,ATH,METALS
34,ICICIPRULI,790.00,-19.62,52.0,H-MC,2.17,138380.0,2592.0,40158.0,1.62,1.91,29.02,31.48,107.0,0.06,0.99,14.26,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.18,53.0,H-LC,1.46,239782.0,-6369.0,85578.0,-3.29,-2.59,35.69,32.18,16.0,-0.07,1.71,32.52,X200,ATH,IT
53,RAJESHEXPO,518.00,1807.83,54.0,L-SC,2.43,52038.0,-85139.0,85233.0,-3.04,-62.07,163.79,0.07,267.0,-1.00,0.37,29.42,OX40N,NTT,JEWELLERY
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.27,82560.0,-30989.0,71043.0,-2.98,-27.29,86.05,35.27,268.0,-0.44,0.59,102.07,XR,NTT,HOTELS
27,HAPPSTMNDS,1488.71,-15.49,43.0,H-SC,10.28,91200.0,-36965.0,146996.0,-1.89,-28.84,161.18,85.85,132.0,-0.25,0.65,5.52,AR,ATH,IT
85,WIPRO,420.00,-11.52,50.0,M-LC,5.67,155676.0,4731.0,104723.0,-1.87,3.13,67.27,72.51,53.0,0.05,1.11,9.35,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,56.0,H-SC,16.77,201691.0,1737.0,95501.0,-0.27,0.87,47.35,48.63,115.0,0.02,1.44,12.52,XR,NTT,MISC
29,HAVELLS,2069.16,0.20,58.0,H-MC,1.93,248368.0,637.0,74411.0,0.08,0.26,29.96,30.30,92.0,0.01,1.77,13.58,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.33,123606.0,-18873.0,38392.0,-0.02,-13.25,31.06,13.70,163.0,-0.49,0.88,45.89,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,102.13,51.0,M-SC,3.30,87076.0,-13701.0,13793.0,0.06,-13.60,15.84,0.09,245.0,-0.99,0.62,14.40,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.24,60.0,H-SC,0.83,225711.0,-43956.0,80285.0,-0.42,-16.30,35.57,13.47,138.0,-0.55,1.61,14.16,XY24,NTT,PAINTS
18,CERA,9475.0,-20.76,42.0,H-SC,2.16,144394.0,-31509.0,73525.0,0.22,-17.91,50.92,23.89,149.0,-0.43,1.03,24.70,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-20.18,51.0,H-MC,7.22,107872.0,-23963.0,66190.0,-0.79,-18.18,61.36,32.03,98.0,-0.36,0.77,20.83,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1258.85,80.0,M-SC,6.42,184906.0,3763.0,41696.0,10.91,2.08,22.55,25.10,235.0,0.09,1.32,48.18,XY24,NTT,HEALTHCARE
59,SAIL,228.00,47.55,67.0,M-MC,8.15,232470.0,7508.0,160149.0,-0.88,3.34,68.89,74.53,192.0,0.05,1.66,36.67,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,14.38,66.0,H-MC,3.48,197320.0,32650.0,10675.0,0.48,19.83,5.41,26.31,89.0,3.06,1.41,29.39,X40N,NTT,AC
17,CAMS,5211.76,1.04,68.0,H-SC,2.99,114915.0,12911.0,31016.0,0.08,12.66,26.99,43.06,122.0,0.42,0.82,31.94,X40N,ATH,MISC
29,HAVELLS,2069.16,0.20,58.0,H-MC,1.93,248368.0,637.0,74411.0,0.08,0.26,29.96,30.30,92.0,0.01,1.77,13.58,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,67.0,H-MC,8.61,112800.0,12296.0,16807.0,0.58,12.23,14.90,28.96,79.0,0.73,0.80,50.36,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.29,53.0,M-SC,0.85,100920.0,9422.0,69332.0,0.58,10.30,68.70,86.07,223.0,0.14,0.72,46.55,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,68.0,H-MC,6.63,186606.0,6798.0,105619.0,0.71,3.78,56.60,62.52,88.0,0.06,1.33,39.16,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.33,33.0,H-SC,11.36,130359.0,7023.0,126735.0,-0.40,5.69,97.22,108.45,119.0,0.06,0.93,29.32,AR,ATH,MISC
85,WIPRO,420.00,-11.52,50.0,M-LC,5.67,155676.0,4731.0,104723.0,-1.87,3.13,67.27,72.51,53.0,0.05,1.11,9.35,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HCLTECH,1943.91,-0.65,43.0,H-LC,9.82,226786.0,-15138.0,78400.0,-1.56,-6.26,34.57,26.15,8.0,-0.19,1.62,7.17,X40,ATH,IT
82,VBL,671.64,-15.98,43.0,H-LC,6.78,302731.0,-13111.0,125785.0,0.01,-4.15,41.55,35.67,5.0,-0.10,2.16,9.13,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.32,46.0,H-LC,7.74,218953.0,-32815.0,86027.0,0.26,-13.03,39.29,21.13,27.0,-0.38,1.56,17.77,X40,ATH,PAINTS
41,ITC,452.00,-37.92,50.0,H-LC,2.24,200612.0,474.0,20422.0,-0.10,0.24,10.18,10.44,4.0,0.02,1.43,5.81,X40,NTT,FMCG
39,INFY,2275.00,-17.09,52.0,H-LC,8.14,322397.0,9509.0,162166.0,-1.73,3.04,50.30,54.87,3.0,0.06,2.30,9.89,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-3.18,53.0,H-LC,1.46,239782.0,-6369.0,85578.0,-3.29,-2.59,35.69,32.18,16.0,-0.07,1.71,32.52,X200,ATH,IT
41,ITC,452.0,-37.92,50.0,H-LC,2.24,200612.0,474.0,20422.0,-0.10,0.24,10.18,10.44,4.0,0.02,1.43,5.81,X40,NTT,FMCG
64,SIEMENS,4671.5,2.95,62.0,H-LC,3.55,165505.0,-20590.0,68072.0,-0.48,-11.06,41.13,25.51,15.0,-0.30,1.18,21.96,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-8.86,54.0,H-LC,4.47,280332.0,14906.0,41882.0,0.29,5.62,14.94,21.40,11.0,0.36,2.00,13.11,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,56.0,H-LC,4.83,219632.0,9266.0,19525.0,0.04,4.40,8.89,13.68,37.0,0.47,1.57,21.66,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.24,200612.0,474.0,20422.0,-0.10,0.24,10.18,10.44,4.0,0.02,1.43,5.81,X40,NTT,FMCG
79,UNITDSPR,1644.00,-10.95,60.0,H-LC,7.60,233318.0,-826.0,49440.0,2.13,-0.35,21.19,20.77,86.0,-0.02,1.66,6.54,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-0.65,43.0,H-LC,9.82,226786.0,-15138.0,78400.0,-1.56,-6.26,34.57,26.15,8.0,-0.19,1.62,7.17,X40,ATH,IT
82,VBL,671.64,-15.98,43.0,H-LC,6.78,302731.0,-13111.0,125785.0,0.01,-4.15,41.55,35.67,5.0,-0.10,2.16,9.13,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,2.95,62.0,H-LC,3.55,165505.0,-20590.0,68072.0,-0.48,-11.06,41.13,25.51,15.0,-0.30,1.18,21.96,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,50.0,H-LC,2.24,200612.0,474.0,20422.0,-0.10,0.24,10.18,10.44,4.0,0.02,1.43,5.81,X40,NTT,FMCG
19,CIPLA,1795.00,-19.44,54.0,H-LC,5.58,215241.0,10741.0,30672.0,-0.30,5.25,14.25,20.25,10.0,0.35,1.54,14.44,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.32,46.0,H-LC,7.74,218953.0,-32815.0,86027.0,0.26,-13.03,39.29,21.13,27.0,-0.38,1.56,17.77,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.83,219632.0,9266.0,19525.0,0.04,4.40,8.89,13.68,37.0,0.47,1.57,21.66,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,58.0,H-LC,12.29,297905.0,-48051.0,122916.0,-0.24,-13.89,41.26,21.64,1.0,-0.39,2.13,5.53,X40,ATH,IT
39,INFY,2275.00,-17.09,52.0,H-LC,8.14,322397.0,9509.0,162166.0,-1.73,3.04,50.30,54.87,3.0,0.06,2.30,9.89,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.24,200612.0,474.0,20422.0,-0.10,0.24,10.18,10.44,4.0,0.02,1.43,5.81,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.78,302731.0,-13111.0,125785.0,0.01,-4.15,41.55,35.67,5.0,-0.10,2.16,9.13,X40N,ATH,FMCG
1,ABB,7934.00,-37.73,66.0,H-LC,9.41,260059.0,-1560.0,120771.0,-0.35,-0.60,46.44,45.57,7.0,-0.01,1.86,10.66,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TRIDENT,48.00,-3.11,69.0,M-SC,1.86,75310.0,-16944.0,43251.0,1.23,-18.37,57.43,28.51,224.0,-0.39,0.54,28.87,XR,NTT,TEXTILES
6,ASIANTILES,137.00,7011.11,64.0,L-SC,5.15,78011.0,-15799.0,92420.0,-0.38,-16.84,118.47,81.67,269.0,-0.17,0.56,51.07,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.27,82560.0,-30989.0,71043.0,-2.98,-27.29,86.05,35.27,268.0,-0.44,0.59,102.07,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,66.0,M-SC,3.92,85008.0,-17202.0,110595.0,-1.07,-16.83,130.10,91.37,208.0,-0.16,0.61,38.00,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-3.67,50.0,H-SC,12.29,89694.0,-11269.0,105803.0,0.38,-11.16,117.96,93.63,148.0,-0.11,0.64,33.60,SR,ATH,CHEMICALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1258.85,80.0,M-SC,6.42,184906.0,3763.0,41696.0,10.91,2.08,22.55,25.10,235.0,0.09,1.32,48.18,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-13.76,79.0,H-SC,13.57,231183.0,-39578.0,369523.0,0.24,-14.62,159.84,121.86,133.0,-0.11,1.65,80.85,AR,ATH,IT
7,ATULAUTO,844.00,3835.14,60.0,M-SC,4.91,122685.0,-24683.0,76494.0,1.12,-16.75,62.35,35.16,236.0,-0.32,0.88,25.31,XY24,NTT,AUTO
60,SAMMAANCAP,326.00,-173.86,66.0,M-SC,3.92,85008.0,-17202.0,110595.0,-1.07,-16.83,130.10,91.37,208.0,-0.16,0.61,38.00,XY25,NTT,FINANCE
26,GREENPANEL,537.00,232.69,61.0,M-SC,1.12,148572.0,-34506.0,113479.0,-0.38,-18.85,76.38,43.14,230.0,-0.30,1.06,39.75,XY24,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.60
1,25,44.18
2,50,75.26


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.97
LC    31.22
MC    23.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.89
X40      14.97
X40N     11.68
XR       11.44
AR        8.60
XY25      8.39
OX40N     7.97
X200      1.71
MH        1.68
X5K       1.43
SR        1.25
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.98
H-LC    24.67
H-MC    20.88
M-SC    13.47
M-LC     5.51
M-MC     2.64
L-SC     1.52
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.23,-13.37,75.18
FMCG,12.32,-3.93,41.12
FINANCE,10.23,-14.41,64.32
MISC,7.25,-13.19,76.13
BANKS,6.21,-14.58,74.85
PAINTS,5.71,-15.46,32.67
ELECTRICAL,5.61,-4.78,41.54
INSURANCE,3.74,-5.52,44.97
AC,3.60,4.11,11.94


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3053349.0,22
XR,1306946.0,14
AR,1125616.0,9
X40,749681.0,10
X40N,568239.0,9
OX40N,530006.0,10
XY25,371923.0,6
SR,245986.0,2
X5K,116996.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3590380.0,29
M-SC,1360387.0,17
H-MC,1177888.0,15
H-LC,1071427.0,14
M-LC,377318.0,4
M-MC,308070.0,2
L-SC,248696.0,3
L-MC,60149.0,1
L-LC,38194.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1205339.0      6
M-SC       XY24       777274.0      7
H-SC       AR         774701.0      5
           XR         760409.0      7
H-MC       XY24       547296.0      4
H-LC       X40        469931.0      5
M-MC       XY24       308070.0      2
H-SC       X40N       290481.0      4
           OX40N      252082.0      4
           SR         245986.0      2
H-MC       X40        208388.0      4
H-LC       X40N       205897.0      3
           AR         188843.0      2
L-SC       XR         163463.0      2
M-SC       AR         162072.0      2
H-MC       XY25       161727.0      2
M-LC       XY24       155599.0      2
M-SC       OX40N      126501.0      4
M-LC       X5K        116996.0      1
M-SC       XR         112583.0      2
           XY25       110595.0      1
H-MC       XR         105619.0      1
M-LC       XR         104723.0      1
H-LC       X200        85578.0      1
L-SC       OX40N       85233.0      1
H-MC       X40N        71861.0      2
M-SC       X40         71362.0      1
H-MC       OX40N       66190.0      1
H-LC       XY25        61407.0      2
H-SC       MH          61382.0      1
L-MC       XR          60149.0      1
H-LC       XY24        59771.0      1
L-LC       XY25        38194.0      1
H-MC       MH          16807.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
